# **Contextual** information is the source of richness and precision with any generative AI work. The right hand (completion/chat) is so popular. But the underappreciated 🫲 left hand is definitely a vital frontier ...

## 🔥 We warm up a kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.1.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Chroma, 1.1.0-alpha"

In [ ]:
// Load settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

In [ ]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052

var memoryBuilder = new MemoryBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "text-embedding-ada-002",
        azureEndpoint, 
        apiKey,
        "model-id");
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);
}

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var memory = memoryBuilder.Build();

# 🃏 Working with context is like a game of chance

## ♦️♥️♣️♠️🃏 First you lay down your cards

In [ ]:
const string MemoryCollectionName = "aboutMeNew";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "I am called Andrea by my friends");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");

## 🤔 And you make a guess in semantic, or "latent," space

In [ ]:
var questions = new[]
{
    "name?"
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + "\n> ... guess: " + response?.Metadata.Text + "\n");
}

# 🃏 This game can be played many ways

###  🔥 First let's get some setup done ...

In [ ]:
const string memoryCollectionName = "SKlearnabilia";

var githubFiles = new Dictionary<string, string>()
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/plugins/ChatPlugin/ChatGPT"]
        = "Sample demonstrating how to create a chat plugin interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/Plugins/Plugins.Memory/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store",
};

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Chroma;
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0022, SKEXP0052

var memoryBuilder = new MemoryBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey, "model-id");
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);
}


## 🃏 There are many versions of this game, OSS or commercial

To run Chroma locally, here's a quick script to download Chroma source and run it using Docker:

```shell
git clone https://github.com/chroma-core/chroma.git
cd chroma
docker-compose up --build
```

In [ ]:
#pragma warning disable SKEXP0011, SKEXP0022, SKEXP0052

var chromaMemoryStore = new ChromaMemoryStore("http://127.0.0.1:8000");

memoryBuilder.WithMemoryStore(chromaMemoryStore);

var memory = memoryBuilder.Build();

In [ ]:
Console.WriteLine("Adding some GitHub file URLs and their descriptions to Chroma.");
var i = 0;
foreach (var entry in githubFiles)
{
    await memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: entry.Value,
        text: entry.Value,
        externalId: entry.Key,
        externalSourceName: "GitHub"
    );
    Console.WriteLine($"  URL {++i} saved");
}

In [ ]:
string ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" +
                    "Query: " + ask + "\n");

var memories = memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.6);

i = 0;
await foreach (var memory in memories)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  URL:     : " + memory.Metadata.Id);
    Console.WriteLine("  Title    : " + memory.Metadata.Description);
    Console.WriteLine("  Relevance: " + memory.Relevance);
    Console.WriteLine();
}

# 💛 A personal favorite for the 🫲 hand game?

It's a tie between [Azure AI Search](https://azure.microsoft.com/en-us/products/ai-services/ai-search) and open-source project [LlamaIndex](https://www.llamaindex.ai/) for sure ...